In [ ]:
using Pkg;
using JuMP, HiGHS, StatsPlots, XLSX; #changed Gurobi to highs

In [ ]:
#DATA IMPORT

function readdata(folder,file)
    #set location for data import & export
    global currentfolder = folder
    global currentcase = file


    #read in data file
    xf = XLSX.readxlsx(currentfolder*currentcase*"test2.xlsx")


    #load data
    l_epb = xf["ep_b"]; #energy purchase prices
    l_eps = xf["ep_s"]; #energy selling prices
    #l_frfc = xf["frfc"];
    l_periods = xf["p"];
    l_ed = xf["ed"];    #energy demand
    l_rg = xf["rg"];   #renewable generation
    l_emin = xf["emin"];
    l_emax = xf["emax"];
    l_einit = xf["einit"];
    #l_cel = xf["cel"];
    l_pch = xf["pch"]; #max charging power
    l_pdis = xf["pdis"]; #max discharging power
    l_cfc = xf["cfc"]; #fuel cell rated capacity (kW) 
    l_nch = xf["nch"]; #charging efficiency
    l_ndis = xf["ndis"]; #discharging efficiency
    l_dr = xf["dr"]; #demand response events******
    l_baseline = xf["baseline"]; #baseline for DR events******

    #PARAMETER AND SET DECLARATION
    #Sets
    global Periods = Int64.(l_periods[:]); #p

    global P = length(Periods); #p

    global edem = Float64.(l_ed[:]);     #energy demand of machine i in state s
    global ß_buy = Float64.(l_epb[:]);    #energy purchase prices in period p
    global ß_sell = Float64.(l_eps[:]);    #energy selling prices in period p
    global re = Float64.(l_rg[:]);    #renewable generation in period p
    
    #DR EVENTS
    global DR_event = Int.(l_dr[:])   # signal DR event, 1 = DR event, 0 = normal period********
    global baseline = Float64.(l_baseline[1,1])   # kW *******
    global π_DR = 18.0   # $/kWh paid by ConEd
   
    #DEMAND CHARGES
    

    #sos2 parameters for efficiency calculation of fuel cell
    #global Breakpoints = [1,2,3,4,5]; #b
    #global B = length(Breakpoints); #b

    #HESS parameter --> convert to batter
    global e_min = Float64.(l_emin[1,1]); #min battery charge
    global e_max = Float64.(l_emax[1,1]); #max battery charge  
    global e_init = Float64.(l_einit[1,1]); #initial battery charge

    global η_ch   = Float64.(l_nch[1,1])    # charging efficiency (–)
    global η_dis  = Float64.(l_ndis[1,1])    # discharging efficiency (–)
    
    
    global p_ch   = Float64.(l_pch[1,1])    # max charging power (kW)
    global p_dis  = Float64.(l_pdis[1,1])   # max discharging power (kW)
    

    #sos2 parameters for efficiency calculation of fuel cell
    #global Breakpoints = [1,2,3,4,5]; #b *****
    #global B = length(Breakpoints); #b ******

    #Fuel Cell
    #global c_FC = Float64.(l_cfc[1,1]); #fuel cell rated capacity (kW)
    #global fclf = [0.00, 0.25, 0.50, 0.75, 1.00]; #fuel cell load factor
    #global r_FC = fclf*c_FC; #electric power output in breakpoints
    #global f_x = [0.01, 0.48, 0.52, 0.50, 0.47]; #fuel cell efficiency factor in breakpoints 
    #global ginfc = r_FC/f_x; #fuel cell gas input

    
end;

In [ ]:
function runmodel()
    
    #BUILD MODEL

    m = Model(HiGHS.Optimizer);
    #set_optimizer_attributes(m, "MIPGap" => 0.01, "TimeLimit" => 180, "PreSOS2BigM" => 1500);
     set_optimizer_attributes(m, "mip_rel_gap" => 0.01, "time_limit" => 180.0);

    #VARIABLES
    #Energy
    @variable(m, E_sold[1:P] >= 0); #p Energy sold in kWh/period in period p
    @variable(m, E_purchased[1:P] >= 0); #p Energy purchased in kWh/period in period p
    
    #Demand Respone Events
    @variable(m, DR_kWh[1:P] >= 0) # DR payment available Demand Response energy reduction in kWh/period in period p*******

#   @variable(m, EDEM[1:P]); #energy demand in period p
    @variable(m, EC); #total energy cost
    #Hydrogen --> Battery
   #@variable(m, R_EL[1:P] >= 0); #power consumption of electrolyzer --> DONT NEED?*
   #@variable(m, R_FC[1:P] >= 0); #hydrogen consumption of fuel cell (in kWh)
   #@variable(m, H[1:P] >= 0); #hydrogen tank fill
    #@variable(m, Λ[1:B,1:P] >= 0); #Λ for linear Interpolation of efficiency curve of fuel cell ********** DO I NEED?******
 
    @variable(m, E_CH[1:P] >= 0); #battery charging (kW)
    @variable(m, E_DIS[1:P] >= 0); #battery discharging (kW)
    @variable(m, SOC[1:P] >= 0); # battery state of charge (kWh)

   #Natural gas fuel cel
    #@variable(m, R_NGFC[1:P] >= 0); # electric output (kW)
    #@variable(m, G_NG[1:P] >= 0); # natural gas input (kWh gas)

 

    #OBJECTIVE function
    @objective(m, Min, EC); 
   
    
    @constraint(m, [p=1:P], e_min <= SOC[p] <= e_max ); #battery capacity  
    @constraint(m, [p=1], SOC[p] == e_init + η_ch*E_CH[p] - (1/η_dis)*E_DIS[p]); #first period battery state of charge
    @constraint(m, [p=2:P], SOC[p] == SOC[p-1] + η_ch*E_CH[p] - (1/η_dis)*E_DIS[p]); #battery state of charge balance
    @constraint(m, [p=1:P], E_CH[p] <= p_ch); #battery charging power limit
    @constraint(m, [p=1:P], E_DIS[p] <= p_dis); #battery discharging power limit

    @constraint(m, [p=1:P], DR_kWh[p] <= DR_event[p] * (baseline - E_purchased[p])) #equals load reduction and can only happen then *************
    @constraint(m, [p=1:P], DR_kWh[p] <= DR_event[p] * baseline)
    
    #@constraint(m, [p=1:P], DR_kWh[p] <= baseline[p] - grid_import[p]) #equals load reduction and can only happen then *************
    #@constraint(m, [p=1:P], DR_kWh[p] <= DR_event[p] * baseline[p]) # only allowed during DR events*************

    #@constraint(m, [p=1:P], G_NG[p] == R_NGFC[p] / f_r_FC[B]); #natural gas consumption calculation --> I believe this is wrong, lower line right

    #@constraint(m, [p=1:P], edem[p] + E_sold[p]+ E_CH[p] == E_purchased[p]+ re[p]+ E_DIS[p] + R_NGFC[p]); #energy balance constraint  
    @constraint(m, [p=1:P], edem[p] + E_sold[p]+ E_CH[p] == E_purchased[p]+ re[p]+ E_DIS[p]); #energy balance constraint  

    #@constraint(m, EC >= sum(E_purchased[p]*ß_buy[p] for p=1:P) - sum(E_sold[p]*ß_sell[p] for p=1:P)); #energy cost calculation
    @constraint(m, EC >= sum(E_purchased[p] * ß_buy[p] for p=1:P)- sum(E_sold[p] * ß_sell[p] for p=1:P) - sum(DR_kWh[p] * π_DR for p=1:P)) #*********

    #@constraint(m, EC == energy_cost - sum(DR_kWh[p] * π_DR[p] for p=1:P))**************

    #@constraint(m, [p=1:P], sum(Λ[b,p] for b in 1:B) == 1) ***********
   #@constraint(m, [p=1:P], SOS2(Λ[:,p]))
    #@constraint(m, [p=1:P], Λ[1:B,p] in MOI.SOS2([1.0,2.0,3.0,4.0,5.0])); #sos2 constraint 2: Only 2 adjacent Breakpoints can be >0 *********


    #@constraint(m, [p=1:P], G_NG[p] == sum(Λ[b,p]*f_r_FC[b] for b=1:B)); #sos2 constraint 4: Fuel Cell Output calculation ***** need for nat gas fuel cell

    #OPTIMIZE model
    optimize!(m);
    
    #variable RETURN
#    solve_time = MOI.get(m, MOI.SolveTime())
#    W = JuMP.value.(W);
#    X = JuMP.value.(X);
#    Y = JuMP.value.(Y);
    #R_EL = JuMP.value.(R_EL);
    #R_FC = JuMP.value.(R_FC);
    #H = JuMP.value.(H);
    EC = JuMP.value.(EC);
#    LC = JuMP.value.(LC);
    E_purchased = JuMP.value.(E_purchased)
    E_sold = JuMP.value.(E_sold)
#    V = JuMP.value.(V)
#    EDEM=JuMP.value.(EDEM)

#Battery
E_CH = JuMP.value.(E_CH);
E_DIS = JuMP.value.(E_DIS);
SOC = JuMP.value.(SOC);

#Fuel Cell
#R_NGFC = JuMP.value.(R_NGFC);
#G_NG = JuMP.value.(G_NG);

  # Λ = JuMP.value.(Λ); *****************
   #F_NGFC = zeros(P)
   for p=1:P, b=1:B
   #    F_NGFC[p] = F_NGFC[p]+(Λ[b,p]*f_r_FC[b]);
    end
    
    #return EC, E_purchased, E_sold, E_CH, E_DIS, SOC, R_NGFC, G_NG,F_NGFC, Λ #ohne solve_time
    return EC, E_purchased, E_sold, E_CH, E_DIS, SOC #ohne solve_time
end;

In [ ]:
#SAVE DATA & GRAPHS

#function savedata(R_NGFC, G_NG, EC, E_purchased, E_sold, F_FC, Λ) #ohne solve_time
function savedata(EC, E_purchased, E_sold, E_CH, E_DIS, SOC) #ohne solve_time

    #save output in excel
    XLSX.openxlsx(currentfolder*currentcase*"test2.xlsx", mode="rw") do xf
        sheet = xf["RESULTS"]
        #sheet["B1"]=R_NGFC #electric output of fuel cell
        #sheet["B2"]=F_NGFC #gas flow from SOS2 (kwh)
        #sheet["B3"]=G_NG #gas input to fuel cell
        sheet["B3"]=SOC #battery SOC
        sheet["B4"]=E_sold #energy sold
        sheet["B5"]=E_purchased #energy purchased
#        sheet["B6"]=V
        sheet["B7"]=EC #total energy cost
        #sheet["B8"]=SOC #battery SOC
#        sheet["B8"]=LC
#        sheet["B9"]=edem
#        sheet["B10"]=solve_time
        println("EC = ", EC)
        end;


end;


In [ ]:
#AUTO INSTANCE CALCULATION
#for g=1:52
    fo="/Users/dbaby/Desktop/"
    fi="Thesis Optimization/"
#    fi_ws="Week"*string(g-1)*"/"
    readdata(fo,fi)
    savedata(runmodel()...)
end
#end;